In [1]:
from z3 import *

In [2]:
# Example problem parameters
m = 3  # Number of couriers
n = 7  # Number of items

# Courier capacities and item sizes
load_limits = [15, 10, 7]
item_sizes = [3, 2, 6, 8, 5, 4, 4]

# Distance matrix between items (including origin point at index n)
distance_matrix = [
    [0, 3, 3, 6, 5, 6, 6, 2],
    [3, 0, 4, 3, 4, 7, 7, 3],
    [3, 4, 0, 7, 6, 3, 5, 3],
    [6, 3, 7, 0, 3, 6, 6, 4],
    [5, 4, 6, 3, 0, 3, 3, 3],
    [6, 7, 3, 6, 3, 0, 2, 4],
    [6, 7, 5, 6, 3, 2, 0, 4],
    [2, 3, 3, 4, 3, 4, 4, 0]
]


In [16]:
# Z3 optimiser initialization
optimizer = Optimize()

In [4]:
# Boolean variables: x[i][j] = True if courier i is assigned item j
x = [[Bool(f"x_{i}_{j}") for j in range(n)] for i in range(m)]

In [5]:
# Variables for the total distance traveled by each courier
total_distance = [Int(f"D_{i}") for i in range(m)]

In [6]:
# Maximum distance traveled by any courier (to minimize)
max_distance = Int('Z')

**Constraints**

In [21]:
# 1. Each item must be assigned to exactly one courier
for j in range(n):
    optimizer.add(Sum([If(x[i][j], 1, 0) for i in range(m)]) == 1)

In [20]:
# 2. The load carried by each courier must not exceed its capacity
for i in range(m):
    optimizer.add(Sum([If(x[i][j], item_sizes[j], 0) for j in range(n)]) <= load_limits[i])


In [19]:
# 3. Distance calculation for each courier based on item assignments
for i in range(m):
    courier_distances = []
    for j in range(n):
        for k in range(n):
            if j != k:
                courier_distances.append(If(And(x[i][j], x[i][k]), distance_matrix[j][k], 0))
    
    # Distance from the origin to the first assigned item and back to origin
    origin_distances = [If(x[i][j], distance_matrix[n][j] + distance_matrix[j][n], 0) for j in range(n)]
    
    optimizer.add(total_distance[i] == Sum(courier_distances) + Sum(origin_distances))

    

In [18]:
# 4. Maximize the minimum distance traveled by any courier
for i in range(m):
    optimizer.add(total_distance[i] <= max_distance)

In [17]:
# 4. Maximize the minimum distance traveled by any courier
# Adding constraints that max_distance is >= total distance for each courier
for i in range(m):
    optimizer.add(max_distance >= total_distance[i])

# Objective: Minimize the maximum distance
optimizer.minimize(max_distance)

In [22]:
# Solve the problem
if optimizer.check() == sat:
    model = optimizer.model()
    print(f"Max distance: {model[max_distance]}")
    
    for i in range(m):
        assigned_items = [j for j in range(n) if model[x[i][j]]]
        print(f"Courier {i+1} is assigned items: {assigned_items} with distance {model[total_distance[i]]}")
else:
    print("No solution found")

Max distance: 40
Courier 1 is assigned items: [1, 3, 4] with distance 40
Courier 2 is assigned items: [2, 5] with distance 20
Courier 3 is assigned items: [0, 6] with distance 24
